In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, concat_ws, collect_list

import pandas as pd
import numpy as np

In [2]:
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/23 10:53:14 INFO SparkEnv: Registering MapOutputTracker
24/04/23 10:53:14 INFO SparkEnv: Registering BlockManagerMaster
24/04/23 10:53:14 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/04/23 10:53:14 INFO SparkEnv: Registering OutputCommitCoordinator


### Export dataset with known IBD drug targets

In [ ]:
# Target-Disease indirect (propagated) evidence from Open Targets Platform
evidence_path = "gs://open-targets-data-releases/23.12/output/etl/parquet/associationByDatatypeIndirect"
evidence = spark.read.parquet(evidence_path)
evidence.persist()

# Filter to only IBD associations
evidence_ibd = evidence.filter(
    (col("diseaseId") == "EFO_0003767") 
).persist()

In [9]:
drug_targets = evidence_ibd.filter(col("datasourceId").contains("chembl"))
drug_targets.count()

275

In [10]:
drug_targets_pd = drug_targets.toPandas()
drug_targets_pd

,datatypeId,datasourceId,diseaseId,targetId,score,evidenceCount
0,known_drug,chembl,EFO_0003767,ENSG00000004779,0.165492,3
1,known_drug,chembl,EFO_0003767,ENSG00000006210,0.143539,3
2,known_drug,chembl,EFO_0003767,ENSG00000007314,0.788195,7
3,known_drug,chembl,EFO_0003767,ENSG00000010671,0.151983,2
4,known_drug,chembl,EFO_0003767,ENSG00000011677,0.870818,7
...,...,...,...,...,...,...
270,known_drug,chembl,EFO_0003767,ENSG00000267855,0.165492,3
271,known_drug,chembl,EFO_0003767,ENSG00000268089,0.607931,1
272,known_drug,chembl,EFO_0003767,ENSG00000273079,0.030397,1
273,known_drug,chembl,EFO_0003767,ENSG00000274286,0.121586,1


In [11]:
drug_targets_pd.to_csv("ibd_drug_targets.csv", index=False)